In [1]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
#import joblib
from pickle import dump

In [2]:
conn = sqlite3.connect('accidentes_usa.db')
cursor_obj = conn.cursor()

# Cargar los conjuntos de datos
X_train_sin_scal = pd.read_sql('SELECT * FROM X_train_sin_scal', conn)
X_test_sin_scal = pd.read_sql('SELECT * FROM X_test_sin_scal', conn)
y_train = pd.read_sql('SELECT * FROM y_train', conn)
y_test = pd.read_sql('SELECT * FROM y_test', conn)

# Cerrar la conexión
conn.close()

In [3]:
X_train_sin_scal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65280 entries, 0 to 65279
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Start_Lat         65280 non-null  float64
 1   Start_Lng         65280 non-null  float64
 2   Traffic_Signal_n  65280 non-null  int64  
 3   Civil_Twilight_n  65280 non-null  int64  
 4   Temperature(F)    65280 non-null  float64
 5   Humidity(%)       65280 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 3.0 MB


In [10]:
X_test_sin_scal.tail(20)

,Start_Lat,Start_Lng,Traffic_Signal_n,Civil_Twilight_n,Temperature(F),Humidity(%)
16300,27.208394,-81.328146,0,1,70.0,100.0
16301,33.668650,-112.117000,0,0,69.0,35.0
16302,40.559439,-74.439675,1,0,31.0,85.0
16303,31.886240,-106.417440,1,0,88.0,14.0
16304,45.495490,-122.565600,0,1,51.0,80.0
16305,34.154880,-118.648891,0,1,43.0,86.0
16306,39.389230,-105.788320,0,0,24.0,81.0
16307,29.228052,-82.186078,0,1,73.0,100.0
16308,39.827310,-104.982850,0,1,52.0,26.0
16309,34.929413,-77.599650,0,1,72.0,94.0


In [11]:
y_test.tail(20)

,Severity
16300,2
16301,3
16302,4
16303,1
16304,2
16305,2
16306,3
16307,2
16308,3
16309,4


In [9]:
X_train_sin_scal['Start_Lng'].describe()


count    65280.000000
mean       -94.550863
std         17.302005
min       -124.474383
25%       -112.099700
50%        -87.919340
75%        -80.163960
max        -68.616840
Name: Start_Lng, dtype: float64

In [4]:
# Convirtiendo la clase en series para correr en los modelos a evaluar
y_train=y_train['Severity']
y_test=y_test['Severity']

In [5]:
# Cargar modelo Random Forest sin escalar para hacer pruebas
from pickle import load
with open("../models/model_randomforestclasiffier_42_sin_scal.sav", "rb") as f:
    model = load(f)

In [6]:
# Hacer predicciones
y_pred = model.predict(X_test_sin_scal)

print(f"Accuracy Score: {np.round(accuracy_score(y_test, y_pred),3)}")
predicciones_exactas = accuracy_score(y_test, y_pred, normalize=False)
print("El número de predicciones exactas es: ", predicciones_exactas) 

Accuracy Score: 0.673
El número de predicciones exactas es:  10990.0


In [ ]:
import streamlit as st
import pandas as pd
from pickle import load
import base64
import os

# Código B
model_path = "C:/Users/gfern/OneDrive/Escritorio/4GeeksAcademy/Proyecto_Final/models/model_randomforestclasiffier_42_sin_scal.sav"
model = load(open(model_path, "rb"))

st.markdown("<h2 style='color: gray;'>Predicción Severidad de Accidentes Viales USA</h2>", unsafe_allow_html=True)
st.write("<p style='color: gray;'>Introducir Valores (Características Tomadas de DataSet Accidentes Viales 2020)</p>", unsafe_allow_html=True)

lat = st.number_input("Latitud", format="%.4f", min_value=24.0000, max_value=50.0000, value=24.0000, step=0.0001)
lng = st.number_input("Longitud", format="%.4f", min_value=-125.0000, max_value=-68.0000, value=-125.0000, step=0.0001)
temp = st.number_input("Temperatura (F)", format="%.2f", min_value=0.00, max_value=110.00, value=0.00, step=0.01)
hmd = st.number_input("Humedad (%)", format="%.2f", min_value=0.00, max_value=100.00, value=0.00, step=0.01)
    
# Usar checkboxes con etiquetas en color black
# Semáforo Cercano
st.markdown("<p style='color: black;'>Semáforo Cercano:</p>", unsafe_allow_html=True)
semaforo = st.radio("Selecciona una opción para Semáforo Cercano:", ["Sí", "No"], 
                    key="semaforo", label_visibility="collapsed")
sem = 1 if semaforo == "Sí" else 0

# Crepúsculo Noche
st.markdown("<p style='color: black;'>Crepúsculo Noche:</p>", unsafe_allow_html=True)
crepusculo = st.radio("Selecciona una opción para Crepúsculo Noche:", ["Sí", "No"], 
                      key="crepusculo", label_visibility="collapsed")
crep = 1 if crepusculo == "Sí" else 0

st.write(f"Semáforo Cercano: {sem}")
st.write(f"Crepúsculo Noche: {crep}")

if st.button("Predecir"):
    data = pd.DataFrame([[float(lat), float(lng), int(sem), int(crep), float(temp), float(hmd)]], 
                            columns=['Start_Lat', 'Start_Lng', 'Traffic_Signal_n', 'Civil_Twilight_n',
                                     'Temperature(F)', 'Humidity(%)'])

    st.write("Datos de entrada:", data)

    prediction = model.predict(data)[0]
    pred_class = f"{prediction:.0f}"

    st.write(f"<p style='color: black;'>La predicción de la Severidad es: {pred_class}</p>", unsafe_allow_html=True)


In [ ]:
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
from pickle import load
import base64

# Configurar el modo ancho automáticamente
#st.set_page_config(layout="wide")


# Función para cargar la imagen de fondo
def get_base64_of_bin_file(bin_file):
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()

# Cargar la imagen de fondo
background_image = get_base64_of_bin_file('Opciones/portada.png')

# Aplicar el estilo CSS para la imagen de fondo y los inputs
page_bg_img = f'''
<style>
.stApp {{
  background-image: url("data:image/png;base64,{background_image}");
  background-size: initial;
}}
</style>
'''
# background-size: unset;

st.markdown(page_bg_img, unsafe_allow_html=True)



In [ ]:
import streamlit as st

# Configurar el modo ancho automáticamente
st.set_page_config(layout="wide")


# Definir las páginas
port = st.Page(
    "Opciones/home.py", title="Home", icon=":material/home:", default=True)

imp = st.Page(
    "Opciones/descrip/import/import.py", title="Importancia del Proyecto", icon=":material/dashboard:")
dat = st.Page(
    "Opciones/descrip/dataset/dataset.py", title="Calidad de los Datos", icon=":material/info:")
sens_mod = st.Page(
    "Opciones/descrip/modelos/sensib.py", title="Optimización de Modelos", icon=":material/settings:")

mod = st.Page("Opciones/modelo.py", title="Modelo Desarrollado", icon=":material/model_training:")

# Crear la navegación
pg = st.navigation(
    {
        "Home": [port],
        "Insights Proyecto": [imp, dat, sens_mod],
        "Modelo Desarrollado": [mod],
    }
)

# Ejecutar la navegación
pg.run()

